In [1]:
!pip install langchain llama-cpp-python chromadb==0.4.14 sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from langchain_community.document_loaders import UnstructuredHTMLLoader, BSHTMLLoader, TextLoader, JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import LlamaCpp
import pickle

In [3]:
from google.colab import drive
import os


drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Capstone')
!ls .

Mounted at /content/drive
'02-19 - Literature Review Mini-Deliverable.gdoc'	     evaluation_set_rag.ipynb
'02-26 - Capstone Project Proposal.gdoc'		     evaluation_set_rag_responses.pickle
'03-04 - Software Tools Mini-Deliverable.gdoc'		     llama-2-7b.Q4_K_M.gguf
'03-11 - Data Leakage Plan Mini-Deliverable.gdoc'	    'LLM Inference Test.ipynb'
'03-11 - First Slack Stand-up Reports and Responses.gdoc'    models
'03-17 - Mentor Check-In.gdoc'				     original_syllabi
'03-18 - Report Outline Mini-Deliverable.gdoc'		     package_installation.ipynb
'03-25 - Second Slack Stand-up Reports and Responses.gdoc'   packages
'03-25 - Visuals Mini-Deliverable.gdoc'			     Pipfile
'04-01 - Revised Outline Mini-Deliverable???.gdoc'	    'Pipfile (1)'
'04-08 - GitHub Repository Mini-Deliverable.gdoc'	     Pipfile.lock
'04-15 - Capstone Project Pre-Flight Checklist.gdoc'	    'Project Tracker.gdoc'
'04-16 - Third Slack Stand-up Reports and Responses.gdoc'    syllabus_loader_embeddings
'04-22 - Final Repor

In [4]:
llm_open = LlamaCpp(
    model_path="./llama-2-7b.Q4_K_M.gguf", # https://huggingface.co/TheBloke/Llama-2-7B-GGUF
    n_ctx=4096,  # 4096 for Llama, 32*1024 for Mistral
    n_gpu_layers=50,
    temperature=0.15,
    top_p=1,
    top_k=40,
    repeat_penalty=1.1,
    max_tokens=1024,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    stream=True,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: -

In [ ]:
def process_llm_response(llm_response):
    print("\n\nSources:")
    for i, source in enumerate(llm_response["source_documents"]):
        m = source.metadata
        print(f"{i + 1}. {m['course_title']} ({m['course_number']}): {m['heading']}")

In [6]:
class ResponseFormatter:
    def __init__(self):
        self.responses = {
            "question": [],
            "contexts": [],
            "answer": [],
            "ground_truth": [],
        }

    def add_response(self, llm_response, ground_truth):
        self.responses["question"].append(llm_response["query"])
        self.responses["contexts"].append(
            [doc.page_content for doc in llm_response["source_documents"]]
        )
        self.responses["answer"].append(llm_response["result"])
        self.responses["ground_truth"].append(ground_truth)

    def get_responses(self):
        return self.responses

In [5]:
query = "Tell me what I need to turn in for the capstone project."

llm_response = llm_open(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 hopefully, it will be a paper and presentation.
I am not sure if you are asking about the Capstone Project or the Capstone Paper. If you are asking about the Capstone Project, then you should contact your instructor. If you are asking about the Capstone Paper, then you can find information on how to write the paper by clicking on the link below:
https://brainmass.com/health-sciences/health-care-management/capstone-project-559841
The Capstone Project is an opportunity for students to demonstrate their ability to integrate their knowledge of health care management into practice. The project is designed to provide students with an opportunity to apply their knowledge of health care management to real-world situations. The project is also designed to help students develop their critical thinking skills and their ability to analyze and synthesize information. The Capstone Project is worth 30% of the final grade. The project is worth 30% of the final grade. The project is worth 30% of the f


llama_print_timings:        load time =    4067.96 ms
llama_print_timings:      sample time =     679.04 ms /  1024 runs   (    0.66 ms per token,  1508.01 tokens per second)
llama_print_timings: prompt eval time =    7761.08 ms /    15 tokens (  517.41 ms per token,     1.93 tokens per second)
llama_print_timings:        eval time =  802815.61 ms /  1023 runs   (  784.77 ms per token,     1.27 tokens per second)
llama_print_timings:       total time =  819321.36 ms /  1038 tokens


In [7]:
llm_response

' hopefully, it will be a paper and presentation.\nI am not sure if you are asking about the Capstone Project or the Capstone Paper. If you are asking about the Capstone Project, then you should contact your instructor. If you are asking about the Capstone Paper, then you can find information on how to write the paper by clicking on the link below:\nhttps://brainmass.com/health-sciences/health-care-management/capstone-project-559841\nThe Capstone Project is an opportunity for students to demonstrate their ability to integrate their knowledge of health care management into practice. The project is designed to provide students with an opportunity to apply their knowledge of health care management to real-world situations. The project is also designed to help students develop their critical thinking skills and their ability to analyze and synthesize information. The Capstone Project is worth 30% of the final grade. The project is worth 30% of the final grade. The project is worth 30% of t

In [8]:
capstone_requirements_llm_response = {"query": query, "result": llm_response, "source_documents": []}


capstone_requirements_ground_truth = """
Final Project Submission including:
    A report that tells the story of your project, which can be formatted as a blog, a scientific manuscript, or something else.
    A GitHub repository full of the code required to reproduce your analysis and figures, documented with an informative README.
    One of the following high-level overviews of your work designed to be shared:
    A 3-5 minute video from your team about what you made.
    A poster appropriate for a data science conference, such as the MIDAS Data Science Symposium.
Weekly Mini-Deliverables: There will be weekly mini-deliverables throughout the course, as outlined in the grading section of the syllabus.
"""

In [9]:
formatter = ResponseFormatter()

In [10]:
formatter.add_response(capstone_requirements_llm_response, capstone_requirements_ground_truth)

with open(f"./evaluation_set_llm_formatter.pickle", 'wb') as handle:
    pickle.dump(formatter, handle)

with open(f"./evaluation_set_llm_responses.pickle", 'wb') as handle:
    pickle.dump(formatter.get_responses(), handle)

In [12]:
prompts = [
    {
        "question": "Which class involves time series analysis?",
        "ground_truth": "Data Mining II, SIADS 632 includes a particular focus on time series analysis.",
    },
    {
        "question": "Who teaches the SQL and Databases class?",
        "ground_truth": "The primary instructor for SQL and Databases is Graham Hukill. Additional course assistants include, Derek Bruckner, Emily Schemanske, Jungseo Lee, and Toby Kemp.",
    },
    {
        "question": "What are the prerequisites for Data Science for Social Good?",
        "ground_truth": "SIADS 630, 631, and 694",
    },
    {
        "question": "When are the office hours for the Math Methods course?",
        "ground_truth": "Office hours for Math Methods are held at the following times: Alex McLeod: Monday at 11:45 am EST, Saurabh Budholiya: Friday at 9:00 am EST, Alexis Castellano: Thursdays at 7:00 pm EST",
    },
    {
        "question": "Are there any weekly readings for Milestone II?",
        "ground_truth": "There is introductory material during the first week of the course, but generally speaking, no weekly readings in this course.",
    },
    {
        "question": "What are the outcomes of Qualitative Inquiry?",
        "ground_truth": "Upon successful completion of this course, students will be able to:1.  Collect, represent, and analyze qualitative data about a quantitative data set, by…2.  Conducting semi-structured interviews;3.  Processing interview notes into discrete pieces of qualitative data; and 4. Analyzing qualitative data using affinity walls.4.  Develop a narrative about qualitative findings that support later quantitative analysis.5.  Communicate qualitative findings in written form.",
    },
]

In [13]:
for prompt in prompts:
  result = llm_open(prompt["question"])

  llm_response = {"query": prompt["question"], "result": result, "source_documents": []}

  formatter.add_response(llm_response, prompt["ground_truth"])

  # Save after each response
  with open(f"./evaluation_set_llm_formatter.pickle", 'wb') as handle:
      pickle.dump(formatter, handle)

  with open(f"./evaluation_set_llm_responses.pickle", 'wb') as handle:
      pickle.dump(formatter.get_responses(), handle)

Llama.generate: prefix-match hit



What is the difference between time series and cross sectional data?
How do you analyze a time series?
What are the types of time series?
What is the best way to forecast time series?
What is time series in statistics?
What is time series analysis in finance?
What is time series analysis in economics?
What is time series analysis in finance with example?
What is time series analysis in economics with example?
What is time series analysis in economics with example?
Time Series Analysis is a branch of econometrics that deals with analyzing time-series data. It is used for analyzing economic, financial, or other time-series data. Time-series data can be used for predicting future values, analyzing trends, or identifying patterns.
Time Series Analysis is a branch of econometrics that deals with analyzing time-series data. It is used for analyzing economic, financial, or other time-series data. Time-series data can be used for predicting future values, analyzing trends, or identifying patt


llama_print_timings:        load time =    4067.96 ms
llama_print_timings:      sample time =     617.22 ms /  1024 runs   (    0.60 ms per token,  1659.05 tokens per second)
llama_print_timings: prompt eval time =    3955.29 ms /     7 tokens (  565.04 ms per token,     1.77 tokens per second)
llama_print_timings:        eval time =  804518.08 ms /  1023 runs   (  786.43 ms per token,     1.27 tokens per second)
llama_print_timings:       total time =  817318.73 ms /  1030 tokens
Llama.generate: prefix-match hit



 surely, it is not a question of who but what.
I'm sure you can find a good book on the subject.
> Who teaches the SQL and Databases class?


llama_print_timings:        load time =    4067.96 ms
llama_print_timings:      sample time =      32.30 ms /    42 runs   (    0.77 ms per token,  1300.27 tokens per second)
llama_print_timings: prompt eval time =    7555.68 ms /    11 tokens (  686.88 ms per token,     1.46 tokens per second)
llama_print_timings:        eval time =   32800.77 ms /    41 runs   (  800.02 ms per token,     1.25 tokens per second)
llama_print_timings:       total time =   40730.44 ms /    52 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    4067.96 ms
llama_print_timings:      sample time =      63.81 ms /    99 runs   (    0.64 ms per token,  1551.38 tokens per second)
llama_print_timings: prompt eval time =    7062.81 ms /    12 tokens (  588.57 ms per token,     1.70 tokens per second)
llama_print_timings:        eval time =   71821.89 ms /    98 runs   (  732.88 ms per token,     1.36 tokens per second)
llama_print_timings:       total time =   79646.44 ms /   110 tokens
Llama.generate: prefix-match hit



 nobody has posted anything yet.
I'm not sure if this is the right place to post, but I thought it would be a good idea to have some sort of discussion about our milestones and how we are doing on them.
I think that we should all try to post something every week so that we can keep track of each other's progress.
If you don't know what your milestones are or what you need to do to complete them, please ask your instructor or TA.


llama_print_timings:        load time =    4067.96 ms
llama_print_timings:      sample time =      69.16 ms /   109 runs   (    0.63 ms per token,  1576.17 tokens per second)
llama_print_timings: prompt eval time =    6762.83 ms /    12 tokens (  563.57 ms per token,     1.77 tokens per second)
llama_print_timings:        eval time =   77684.95 ms /   108 runs   (  719.31 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =   85273.17 ms /   120 tokens
Llama.generate: prefix-match hit



 surely, you can't just say "I am a qualitative researcher" and then go on to do quantitative research.
The outcomes of qualitative inquiry are not limited to the data that is collected but also include the process of collecting it. The process of collecting data in qualitative research is often more important than the actual data itself. This is because it allows for a deeper understanding of the subject matter being studied.
Qualitative researchers often use interviews or focus groups as their primary method for collecting data. These methods allow for in-depth conversations with participants which can provide valuable insights into their thoughts and feelings about a particular topic or issue. Additionally, these methods allow for open-ended questions which allow participants to share their experiences in their own words rather than having them respond to predetermined questions set by the researcher. This allows for more nuanced understanding of complex issues than would be possib


llama_print_timings:        load time =    4067.96 ms
llama_print_timings:      sample time =     709.87 ms /  1024 runs   (    0.69 ms per token,  1442.51 tokens per second)
llama_print_timings: prompt eval time =    8591.27 ms /    12 tokens (  715.94 ms per token,     1.40 tokens per second)
llama_print_timings:        eval time =  795203.09 ms /  1023 runs   (  777.32 ms per token,     1.29 tokens per second)
llama_print_timings:       total time =  812791.47 ms /  1035 tokens
